# <center>Assignment 14

Name: Alua Bilakhanova (211107012)

In [1]:
# A dictionary of movie critics and their ratings of a small
# set of movies

critics={'Lisa Rose': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.5,
'Just My Luck': 3.0, 'Superman Returns': 3.5, 'You, Me and Dupree': 2.5,
'The Night Listener': 3.0},
        'Gene Seymour': {'Lady in the Water': 3.0, 'Snakes on a Plane': 3.5,
'Just My Luck': 1.5, 'Superman Returns': 5.0, 'The Night Listener': 3.0,
'You, Me and Dupree': 3.5},
        'Michael Phillips': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.0,
'Superman Returns': 3.5, 'The Night Listener': 4.0},
        'Claudia Puig': {'Snakes on a Plane': 3.5, 'Just My Luck': 3.0,
'The Night Listener': 4.5, 'Superman Returns': 4.0,
'You, Me and Dupree': 2.5},
        'Mick LaSalle': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0,
'Just My Luck': 2.0, 'Superman Returns': 3.0, 'The Night Listener': 3.0,
'You, Me and Dupree': 2.0},
        'Jack Matthews': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0,
'The Night Listener': 3.0, 'Superman Returns': 5.0, 'You, Me and Dupree': 3.5},
        'Toby': {'Snakes on a Plane':4.5,'You, Me and Dupree':1.0,'Superman Returns':4.0},
        'Jack John': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0,'The Night Listener': 3.0}}


### Exercises

<b>Ex1</b><br>
1. Tanimoto score. Find out what a Tanimoto similarity score is. In what cases could this be used as the similarity metric instead of Euclidean distance or Pearson coefficient? Create a new similarity function using the Tanimoto score.

Tanimoto coefficient is a metric (or score) to measure the similarity of two sets of elements.

Tanimoto coefficient can be simply defined as the ratio of the intersection of the two sets over the union of the two sets.
More precisely, the Tanimoto coefficient of set A and set B can be defined as:

T = Nc / (Na + Nb - Nc)<br>
where:<br>
  Na is the number of elements in set А<br>
  Nb is the number of elements in set B<br>
  Nc is the number of elements that are shared in A and B<br>

In [2]:
#  similarity function using the Tanimoto score.
def sim_tanimoto(prefs,person1,person2):
    number_prefs_person1 = len(prefs[person1])
    number_prefs_person2 = len(prefs[person2])
    shared_prefs = {}
    for item in prefs[person1]:
        if item in prefs[person2]:
            shared_prefs[item] = 1
    if len(shared_prefs) == 0:
        return 0
    else:
        return len(shared_prefs)/(number_prefs_person1+number_prefs_person2-len(shared_prefs))
    

In [3]:
person1 = 'Lisa Rose'
person2 = 'Gene Seymour'
similarity_score = sim_tanimoto(critics, person1, person2)
print(f'Similarity score of {person1} and {person2} is {similarity_score}')

Similarity score of Lisa Rose and Gene Seymour is 1.0


<b>Ex2</b><br>
   2.  User-based efficiency. The user-based filtering algorithm is inefficient because it compares a user to all other users every time a recommendation is needed. Write a function to precompute user similarities, and alter the recommendation code to use only the top five other users to get recommendations.

In [4]:
# function to precompute user similarities
def precompute_user_similarities(prefs,person):
    similarities = {}
    for other in prefs:
        if other==person: continue
        sim = sim_tanimoto(prefs,person,other)
        similarities[other] = sim
    return similarities
    

In [5]:
d = precompute_user_similarities(critics,person1)
d

{'Gene Seymour': 1.0,
 'Michael Phillips': 0.6666666666666666,
 'Claudia Puig': 0.8333333333333334,
 'Mick LaSalle': 1.0,
 'Jack Matthews': 0.8333333333333334,
 'Toby': 0.5,
 'Jack John': 0.5}

In [6]:
import operator
dict(sorted(d.items(), key=operator.itemgetter(1), reverse=True)[:5])

{'Gene Seymour': 1.0,
 'Mick LaSalle': 1.0,
 'Claudia Puig': 0.8333333333333334,
 'Jack Matthews': 0.8333333333333334,
 'Michael Phillips': 0.6666666666666666}

In [7]:
def getRecommendations(prefs,person,similarity=precompute_user_similarities):
    totals={}
    simSums={}
    d = similarity(prefs,person)
    #Top five others
    top_five_others = dict(sorted(d.items(), key=operator.itemgetter(1), reverse=True)[:5])
    
    for other in top_five_others.keys():
        if other==person: continue
        sim = top_five_others[other]
        if sim<=0: continue
        for item in prefs[other]:
            if item not in prefs[person] or prefs[person][item]==0:
                totals.setdefault(item,0)
                totals[item]+=prefs[other][item]*sim
                simSums.setdefault(item,0)
                simSums[item]+=sim
                
    rankings=[(total/simSums[item],item) for item,total in totals.items( )]
    rankings.sort( )
    rankings.reverse( )
    return rankings

In [8]:
getRecommendations(critics,'Toby')

[(3.333333333333333, 'The Night Listener'),
 (2.880952380952381, 'Lady in the Water'),
 (2.4047619047619047, 'Just My Luck')]